### Import library
Nous allons importer les bibliothèque numpy et pandas qui sont necessaire pour le traitement.

In [2]:
import pandas as pd
import numpy as np

### Import Data
Nous allons importer les données depuis le répertoire RAwData pour enfin les traiter et transformer

In [6]:
n=['TimesTamp', 'ECP', 'Antenna', 'RSSI', 'FP']
p='../1-RawData/datasetfinale/mergedFiles/all-3/p25.csv'
d=','
def importData(path, delimit,cols):
    return pd.read_csv(path, sep=delimit,names=cols)

data=importData(p,d,n)

print(data.shape)
data.head(10)

(5925, 5)


,TimesTamp,ECP,Antenna,RSSI,FP
0,1581420971718832,3005FB63AC1F3681EC88149C21,1,-62.5,0
1,1581420971731907,2005FB63AC1F3681AB1004E321,1,-66.5,0
2,1581420971761284,3005FB63AC1F3681EC88149C21,1,-62.5,0
3,1581420972009461,2005FB63AC1F3681AB1004E321,1,-65.0,0
4,1581420972227990,3005FB63AC1F3681EC88149C21,1,-59.5,0
5,1581420972256745,3005FB63AC1F3681EC8812DB21,1,-59.0,0
6,1581420972298026,3005FB63AC1F3681EC8812DB21,1,-58.0,0
7,1581420972513159,3005FB63AC1F3681EC8812DB21,1,-58.0,0
8,1581420972528325,3005FB63AC1F3681EC88149021,1,-62.5,0
9,1581420972550754,3005FB63AC1F3681EC8811B021,1,-58.5,0


### TYPAGE DES CHAMPS
Nos allons typer les différentes colonnes du dataSet avec les types primaires de python 

In [7]:
def typage(data):
    data['ECP']=data['ECP'].astype(str)
    data['TimesTamp']=data['TimesTamp'].astype('int64')
    data['RSSI']=data['RSSI'].astype('float64')
    data['Antenna']=data['Antenna'].astype('int64')
    data['FP']=data['FP'].astype('int64')
    return data

data=typage(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5925 entries, 0 to 5924
Data columns (total 5 columns):
TimesTamp    5925 non-null int64
ECP          5925 non-null object
Antenna      5925 non-null int64
RSSI         5925 non-null float64
FP           5925 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 231.5+ KB


### Build DataSet
Nous allons contruire la dataFrame avec les colonnes

In [20]:
grouped_df = data.groupby('ECP')
print(len(grouped_df))

1129


In [19]:
from scipy.stats import variation 

def generateDataSet(data, groupedBy):
    grouped_df = data.groupby(groupedBy) #On groupe les données par ..."Etiquetes"
    
    dataSet = pd.DataFrame({}) #Création d'une nouvelle dataFrame
    
    #Création des variables intermédiaire pour la récupération des champs
    ecp=pd.Series(grouped_df.ECP.unique().index,name="ECP")
    rc= pd.Series(grouped_df.ECP.count(),name="readcount")
    minRSSI= pd.Series(grouped_df.RSSI.min(),name="minRssi")
    maxRSSI= pd.Series(grouped_df.RSSI.max(),name="maxRssi")
    mean=pd.Series(grouped_df.RSSI.mean(),name="meanRssi")
    startTime=pd.Series(grouped_df.TimesTamp.min(),name="startTime")
    endTime=pd.Series(grouped_df.TimesTamp.max(),name="endTime")
    diff=pd.Series(grouped_df.RSSI.diff,name="difference")
    
    #Mise à jour des champs de la dataFrame
    dataSet['ECP']=ecp.values
    dataSet['RC']=rc.values
    dataSet['MIN']=minRSSI.values
    dataSet['MAX']=maxRSSI.values
    dataSet['MEAN']=mean.values
    dataSet['START']=startTime.values
    dataSet['END']=endTime.values
    dataSet['DURATION']=endTime.values-startTime.values
    cv={}
    a1={}
    a2={}
    a3={}
    a4={}
    #for key, item in grouped_df:
    #    cv[key]=variation(grouped_df['RSSI'].get_group(key).values,axis=0)
    #    a1[key]=0
    #    a2[key]=0
    #    a3[key]=0
    #    a4[key]=0
        #for a in grouped_df['Antenna'].get_group(key).values:
        #    if(a==1): a1[key]+=1
        #    if(a==2): a2[key]+=1
        #    if(a==3): a3[key]+=1
        #    if(a==4): a4[key]+=1
                
    #dataSet['CV']=pd.Series(cv).values
    #dataSet['A1']=pd.Series(a1).values
    #dataSet['A2']=pd.Series(a2).values
    #dataSet['A3']=pd.Series(a3).values
    #dataSet['A4']=pd.Series(a4).values
    
    return dataSet

dataSet=generateDataSet(data,'ECP')
print(dataSet.size)
dataSet.head(10)

9032


,ECP,RC,MIN,MAX,MEAN,START,END,DURATION
0,1007CEA7AC1F3681EAC5106D21,6,-57.0,-49.5,-53.500000,1581420973274508,1581420974357643,1083135
1,1007CEA7AC1F3681EAC5106D22,1,-62.0,-62.0,-62.000000,1581420688744448,1581420688744448,0
2,1007CEA7AC1F3681EAC5106D23,6,-56.0,-48.5,-53.083333,1581420883774374,1581420884548849,774475
3,1007CEA7AC1F3681EAC5106D25,4,-54.0,-45.0,-48.875000,1581420217848224,1581420218371387,523163
4,1007CEA7AC1F3681EAC5106D26,7,-60.0,-45.0,-51.571429,1581420117430720,1581420118848541,1417821
5,1007CEA7AC1F3681EAC5106D29,2,-59.5,-58.5,-59.000000,1581419787766006,1581419787836176,70170
6,1007CEA7AC1F3681EAC5106D30,1,-60.0,-60.0,-60.000000,1581414773721288,1581414773721288,0
7,1007CEA7AC1F3681EAC5106D31,8,-59.5,-53.5,-56.500000,1581415046776906,1581415047793641,1016735
8,1007CEA7AC1F3681EAC5106D32,8,-54.0,-39.0,-47.562500,1581414922500972,1581414923792277,1291305
9,1007CEA7AC1F3681EAC5106D33,16,-60.0,-50.0,-55.781250,1581414578916990,1581414581922373,3005383


### Export Data
Nous allons sauvegarder les donées dans un fichier .csv

In [11]:
import time
def exportData(data, path):
    data.to_csv(path, index = None, header=False)
    

## Export allInOne
Exporter l'ensemble des daonnées dans un seul.

In [12]:
tm = time.strftime("%Y-%b-%d-%H_%M_%S")
path='../2-TransformedData/dataFinale/p25/dataSet_p25_'+tm+'.csv'
exportData(dataSet, path)

### Split Data
Nous allons découper les données en deux, 80% des trainSets et 20% des tesSets

In [13]:
def splitDataLE20(data):
    dataSet_np = np.random.rand(len(data)) <0.8
    trainSet=data[dataSet_np]
    testSet=data[~dataSet_np]
    return trainSet, testSet

from sklearn.model_selection import train_test_split
def sliptDataE20(data):
    X = data.iloc[:, :-1].values
    y = data['FP'].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    trainSet=pd.DataFrame(X_train)
    trainSet[13]=y_train
    
    testSet = pd.DataFrame(X_test)
    testSet[13] = y_test
    return trainSet, testSet

### Export Train & Test
Nous allons exporter les données découpées en traint et test

In [80]:
tm = time.strftime("%Y-%b-%d-%H_%M_%S")
def importData2(path):
    #columnNames=["ECP","RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4","FP"]
    columnNames=["RC","MIN","MAX","MEAN","START","END","DURATION","CV","A1","A2","A3","A4","FP"]
    delimit=','
    data = pd.read_csv(path, names=columnNames, sep=delimit)
    #data = data.drop(columns="ECP")
    return data
you_data=importData2('../2-TransformedData/dataSets_you.csv')

trainSet, testSet = sliptDataE20(you_data)
#trainSet, testSet = sliptDataE20(dataSet)
#trainSet, testSet = splitDataLE20(dataSet)

pathTrain='../3-DataSet/trainSet_you_'+tm+'.csv'
exportData(dataSet, pathTrain)

pathTest='../3-DataSet/testSet_you_'+tm+'.csv'
exportData(dataSet, pathTest)